In [1]:
# Run this block first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Air and moisture transfer

## Exercise 1

The building has two opposite openings: one on the bottom and one on the top. They are separated by a difference of elevation $\Delta z$. The wind blows in the direction shown on the figure. The indoor temperature is 20°C and the outdoor 0°C. The wind pressure coefficients of the top and bottom openings are 0.6 and -0.4, respectively.

<img src="figures/air_ex1.png" alt="ex1" width="250"/>

The wind speed is 5 m/s and the elevation $\Delta z$ is 10 m. Using this equation, calculate which of the two forces is stronger, and show the direction of the air flow inside the building.

$$ Q=A^* \left( \sqrt{\left|\Delta C_p\right|\cdot U_\mathit{wind}^2  } \pm \sqrt{2\cdot g\cdot H \left(\frac{|\Delta T|}{T_\mathit{mean}}\right) } \right) $$

### Solution

In [2]:
# Wind force
U_wind = 5
cp1 = 0.6
cp2 = -0.4
V_wind = np.sqrt((cp1-cp2)*U_wind**2)

# Stack effect
H = 10
T1 = 20 + 273.15
T2 = 0 + 273.15
V_stack = np.sqrt(2*9.8*H*(T1-T2)/((T1+T2)/2))

print("Air speed from wind: ", V_wind)
print("Air speed from stack effect: ", V_stack)

Air speed from wind:  5.0
Air speed from stack effect:  3.7207864979281466


## Exercise 2

We are studying a solar chimney shown below. The air is heated inside the chimney, which creates a stack effect that drives it upwards. The pressure drop creates an air flow into the house, which first goes through a ground-coupled heat exchanger. The target is to calculate the indoor temperature as a consequence of this cold air flow.

<img src="figures/air_ex2.png" alt="ex2" width="250"/>

Inside the chimney, air circulates between a glass collector and a concrete slab. In order to solve the problem, we need at least three equations:
* Write the heat balance of the main room of the house (not the chimney) as function of its global heat transfer coefficient $H=100$ W/K and incoming air flow rate $\dot{m}$.
* Formulate the mass flow rate as function of the air temperatures at the inlet $T_s=13$°C and at the outlet of the chimney $T_\mathrm{out}$, and the difference of elevation $z$ between them.
* We will assume that the maximum temperature reached by the air in the chimney is 45°C.

The thermal properties of the chimney and convective heat transfer coefficients can be taken from previous exercises.

### Solution

The heat balance equation of the room is: 
$$ \dot{m}c_p(T_s-T_i) + H(T_e-T_i) = 0$$

In order to calculate the mass flow rate $\dot{m}$, we first use the following formula for the volumetric flow rate in presence of stack effect, while no wind is considered:
$$ Q=A^* \sqrt{2\cdot g\cdot z \left(\frac{|\Delta T|}{T_\mathit{mean}}\right) } $$
where A$^*$ is the equivalent surface of the openings, $z$ is the evelation difference between inlet and outlet. The temperature difference $\Delta T$ and mean $T_\mathit{mean}$ are formulated relatively to the inlet temperature $T_s$ and the outlet of the chimney $T_\mathrm{out}$. This means that this outlet temperature is required to calculate the air flow rate.

In [11]:
# Calculation of the flow rate
A = 0.06 # we assume this value for the equivalent surface of the opening
g = 9.8
z = 10
T_inlet = 13
T_outlet = 45
Q = A * np.sqrt(2*g*z*(T_outlet-T_inlet) / ((T_inlet+T_outlet)/2+273.15) ) # volumetric flow rate
rho = 1.2
mdot = rho * Q  # mass flow rate
print("mass flow rate: ", mdot)

mass flow rate:  0.32803805028666444


Now we can calculate the indoor temperature from the first heat balance equation.

In [12]:
T_ext = 35
H = 100
cp = 1000

T_indoor = 1/(mdot*cp+H) * (mdot*cp*T_inlet + H*T_ext)
print("Indoor temperature: ", T_indoor)

Indoor temperature:  18.13972998084311


## Exercise 3: condensation on a window

A window separates a room at $T_i=20$°C from the outside at $T_e=8$°C. The window has a thickness of 4 mm and conductivity 1.1 W.m$^{-1}$.K$^{-1}$. The indoor and outdoor superficial resistances are 0.13 and 0.04 m$^2$.K.W$^{-1}$, respectively.

* Calculate the temperature at the inside surface of the window.
* Calculate the saturation vapour pressure at the temperatures of the room and window surface.
* What is the relative humidity at which condensation appears at the window?

### Solution

The surface temperature $T_s$ is so that:
$$ \frac{1}{0.13} \cdot (T_i-T_s) = \frac{1}{0.13+\frac{0.04}{1.1}+0.04} \cdot (T_i-T_e) $$

In [5]:
Ti = 20
Te = 8

Rsi = 0.13  # indoor superficial resistance
Rwin = 0.04 / 1.1  # thermal resistance of the window
Rse = 0.04  # outdoor superficial resistance

Tsurf = Ti - Rsi/(Rse+Rwin+Rsi) * (Ti-Te)

print("Surface temperature: ", Tsurf)

Surface temperature:  12.440528634361232


The saturation vapour pressure is a function of temperature:
$$ p_\mathit{sat}(T) = \mathrm{exp} \left(23.5771-\frac{4042.9}{T+273.15-37.58}\right) $$

In [6]:
def psat(T):
    return np.exp(23.5771 - 4042.9/(T+273.15-37.58))

print("Saturation pressure at room temperature: ", psat(Ti))
print("Saturation pressure at surface temperature: ", psat(Tsurf))

Saturation pressure at room temperature:  2340.1039900862156
Saturation pressure at surface temperature:  1444.8718769780508


Condensation occurs at the window if its vapour pressure $p_v=\mathrm{RH} p_\mathit{sat}$ reaches the saturation pressure. Since we assume that this vapour pressure is the same in the entire room (indoor and surface), the indoor relative humidity where this occurs is the ratio between the two previous values:

In [7]:
print("RH for condensation: ", psat(Tsurf)/psat(Ti))

RH for condensation:  0.6174391749679542


## Exercise 4: condensation in a wall

<img src="figures/air_ex4.png" alt="ex4" width="250"/>

A wall has three layers: concrete, insulation and gypsum. The thickness, thermal conductivity and vapour permeability of each layer is given in the following table, along with the surface transfer coefficient for heat $h_t$ and moisture $h_m$ on each side of the wall.

| Material | $e$ (m) | $\lambda$ (W.m$^{-1}.$K$^{-1}$) | $\delta_\mathrm{vap}$ (s) |
|----------|---------|---------------------------------|---------------------------|
| Concrete | 0.18 | 2 | $2.2\times 10^{-12}$ |
| Insulation | 0.15 | 0.04 | $1.5\times 10^{-10}$ |
| Gypsum | 0.013 | 0.35 | $9\times 10^{-12}$ |

* Outside transfer coefficients: $h_t=16$ (W.m$^{-2}.$K$^{-1}$) and $h_m=4\times10^{-8}$ (s.m$^{-1}$)
* Inside transfer coefficients: $h_t=4.1$ (W.m$^{-2}.$K$^{-1}$) and $h_m=1\times10^{-8}$ (s.m$^{-1}$)

Indoor conditions: $T_i=20$°C and RH$_i$=60$\%$. Outdoor conditions: $T_e=6$°C and RH$_e$=80$\%$.

1. Calculate the vapour pressure $p_v$ of air inside and outside.
2. Calculate the temperature, saturation pressure and vapour pressure at each interface of the wall
3. Is there condensation somewhere?

### Solution

We can define the *moisture transfer resistance* of a wall component as $R=e/\delta_\mathrm{vap}$. The moisture exchange between the surface of a wall and the ambient air can also be formulated in the same way as heat transfer, with a surface moisture transfer coefficient $h_m$:
$$ g_\mathrm{vap} = h_m (p_{v,\mathrm{surf}}-p_{v,\mathrm{air}}) $$

In [8]:
# Conditions
Ti = 20
RHi = 0.6
Te = 6
RHe = 0.8

# Saturation vapour pressure
def psat(T):
    return np.exp(23.5771 - 4042.9/(T+273.15-37.58))

# Vapour pressure of the inside and outside
pvi = RHi * psat(Ti)
pve = RHe * psat(Te)

# Array of thermal resistances
R_th = np.array([1/16, 0.18/2, 0.15/0.04, 0.013/0.35, 1/4.1])
# Array of moisture transfer resistances
R_m = np.array([1/(4e-8), 0.18/(2.2e-12), 0.15/(1.5e-10), 0.013/(9e-12), 1/(1e-8)])

# Calculation of the total heat and moisture flux across the wall
Phi_heat = (Ti - Te) / np.sum(R_th)
Phi_moist = (pvi - pve) / np.sum(R_m)

# Calculation of the 4 temperatures and vapour pressures at the interfaces
T = np.concatenate((np.array([Te]), np.zeros(4), np.array([Ti])))
pv = np.concatenate((np.array([pve]), np.zeros(4), np.array([pvi])))

for i in range(4):
    T[i+1] = T[i] + R_th[i] * Phi_heat
    pv[i+1] = pv[i] + R_m[i] * Phi_moist

print("Distribution of temperatures: ", T)
print("Distribution of vapour pressures: ", pv)
print("Distribution of saturation pressures: ", psat(T))

Distribution of temperatures:  [ 6.          6.20915275  6.51033271 19.05949766 19.18379415 20.        ]
Distribution of vapour pressures:  [ 748.46219513  748.65641798 1384.29482435 1392.06373821 1403.28550267
 1404.06239405]
Distribution of saturation pressures:  [ 935.57774391  949.22112966  969.17528763 2207.29014141 2224.45598692
 2340.10399009]


We can see that there is a point where the vapour pressure exceeds the saturation pressure: this generates condensation.